# The Query Feature

In [2]:
import mysql.connector
import pandas as pd

# For Query Screen

mydatabase = mysql.connector.connect(
    host="localhost",
    user="root",
    password="sid34",
    database="siddata"
)

mycursor = mydatabase.cursor()

mycursor.execute("SELECT * FROM client;")

myresult = mycursor.fetchall()

# printing the result

# for x in myresult:
#     print(x)

df = pd.read_sql_query('SELECT * FROM client;', mydatabase)

print(df)

   client_id          client_name  branch_id
0        402                FedEx          3
1        403  John Daliy Law, LLC          3
2        404  Scranton Whitepages          2
3        405      Times Newspaper          3
4        406                FedEx          2


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23236\73559188.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('SELECT * FROM client', mydatabase)


# Upload To S3 AWS

In [1]:
import boto3

# Get your AWS credentials
ACCESS_KEY_ID = "AKYZH"
SECRET_ACCESS_KEY = "F7O"

# Create a Boto3 client
s3 = boto3.client("s3", aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

# Upload a file to S3
file_path = r"./Storage/sqlBackup_2023-05-07.sql"
bucket_name = "sidmybucket1234"
object_key = "sqlBackup_2023-05-07.sql"

s3.upload_file(file_path, bucket_name, object_key)

# Check if the file is uploaded successfully
response = s3.head_object(Bucket=bucket_name, Key=object_key)

if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
    print("File uploaded successfully")
else:
    print("File upload failed")

File uploaded successfully


# Upload To Google Drive

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file('./data-store-1789-c5507a0628a0.json')

drive_service = build('drive', 'v3', credentials=credentials)

file_path = './Storage/sqlBackup_2023-05-08.sql'
# file_metadata = {
#     'name': 'My File SQL'
# }

media = MediaFileUpload(file_path)
file = drive_service.files().create(media_body=media, fields='id').execute()
print('File ID:', file.get('id'))

# Encryt the files

In [8]:
from cryptography.fernet import Fernet
import os

# Generate a random key
key = Fernet.generate_key()

filepath = "key.txt"

# Checking wether key.txt exists
if(os.path.isfile(filepath)):
    i = 1
    filepath = f"key{i}.txt"
    while(os.path.isfile(filepath)):
        i = i + 1
        filepath = f"key{i}.txt"

# Save the key to a file
with open(filepath, "wb") as f:
    f.write(key)

# Load the key from a file
with open(filepath, "rb") as f:
    key = f.read()

# Create a Fernet object
fernet = Fernet(key)

# opening the original file to encrypt
with open('./Storage/sqlBackup_2023-05-19.sql', 'rb') as file:
    original = file.read()

# encrypting the file
encrypted = fernet.encrypt(original)

# opening the file in write mode and
# writing the encrypted data
with open('./Storage/sqlBackup_2023-05-19.sql', 'wb') as encrypted_file:
    encrypted_file.write(encrypted)


# Decrypt the files

In [9]:
# Select key file first to decrypt

with open("key2.txt", "rb") as f:
    key = f.read()

# Create a Fernet object
fernet = Fernet(key)

# opening the encrypted file
with open('./Storage/sqlBackup_2023-05-19.sql', 'rb') as enc_file:
    encrypted = enc_file.read()

# decrypting the file
decrypted = fernet.decrypt(encrypted)

# opening the file in write mode and
# writing the decrypted data
with open('./Storage/sqlBackup_2023-05-19.sql', 'wb') as dec_file:
    dec_file.write(decrypted)

# The Convert Code

In [ ]:
# SQL TO Mongo

import sqlparse
from pymongo import MongoClient

# Connect to the MongoDB database
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['your_mongodb_database']
mongo_collection = mongo_db['your_mongodb_collection']

# Read and parse the SQL backup file
sql_file_path = '/path/to/sql/backup/file.sql'
with open(sql_file_path, 'r') as sql_file:
    sql_statements = sqlparse.split(sql_file.read())

# Transform and insert data into MongoDB collection
for statement in sql_statements:
    # Skip non-insert statements if needed
    if not sqlparse.sql.Statement(statement).get_type() == 'INSERT':
        continue

    # Parse the SQL insert statement
    parsed = sqlparse.parse(statement)[0]
    table_name = parsed.get_real_name()
    columns = [token.get_real_name() for token in parsed[2].get_identifiers()]
    values = [token.get_real_value() for token in parsed[4].get_identifiers()]

    # Transform values into a MongoDB document format
    doc = dict(zip(columns, values))

    # Insert the document into MongoDB collection
    mongo_collection.insert_one(doc)

# Close the MongoDB connection
mongo_client.close()
